In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time

# Función para limpiar los nombres de las hojas
def clean_sheet_name(name):
    return re.sub(r'[\/:*?"<>|]', '_', name)

# Función para formatear la fecha de estreno
def format_release_date(date_str):
    try:
        formatted_date = pd.to_datetime(date_str, errors='coerce')
        if pd.isnull(formatted_date):
            return None
        return formatted_date.strftime('%Y-%m-%d')
    except Exception as e:
        return None

# Función para extraer detalles adicionales de la página de la película
def get_movie_details(movie_url):
    movie_details = {}
    try:
        movie_response = requests.get(movie_url, timeout=10)
        movie_response.raise_for_status()
        movie_soup = BeautifulSoup(movie_response.text, 'html.parser')

        domestic_distributor = movie_soup.find('span', text='Domestic Distributor')
        if domestic_distributor:
            movie_details['Domestic Distributor'] = domestic_distributor.find_next_sibling('span').text.strip()

        domestic_opening = movie_soup.find('span', text='Domestic Opening')
        if domestic_opening:
            movie_details['Domestic Opening'] = domestic_opening.find_next_sibling('span').text.strip()

        budget = movie_soup.find('span', text='Budget')
        if budget:
            movie_details['Budget'] = budget.find_next_sibling('span').text.strip()

        earliest_release_date = movie_soup.find('span', text='Earliest Release Date')
        if earliest_release_date:
            movie_details['Earliest Release Date'] = earliest_release_date.find_next_sibling('span').text.strip()

        mpaa = movie_soup.find('span', text='MPAA')
        if mpaa:
            movie_details['MPAA'] = mpaa.find_next_sibling('span').text.strip()

        running_time = movie_soup.find('span', text='Running Time')
        if running_time:
            movie_details['Running Time'] = running_time.find_next_sibling('span').text.strip()

        genres = movie_soup.find('span', text='Genres')
        if genres:
            movie_details['Genres'] = genres.find_next_sibling('span').text.strip()

    except Exception as e:
        print(f"Error al obtener detalles de la película: {e}")

    return movie_details

# URL de la página de Top Lifetime Gross en Box Office Mojo
url = "https://www.boxofficemojo.com/chart/ww_top_lifetime_gross/?area=XWW"

# Realizar la solicitud HTTP a la página principal
response = requests.get(url)
response.raise_for_status()
soup = BeautifulSoup(response.text, 'html.parser')

# Extraer la tabla que contiene las películas
movie_table = soup.find('table')

# Almacenar los detalles de cada película
movie_details = []

if movie_table:
    movie_rows = movie_table.find_all('tr')[1:]  # Ignorar la fila de encabezado

    for movie_row in movie_rows:
        movie_cells = movie_row.find_all('td')
        if len(movie_cells) >= 6:
            rank = movie_cells[0].text.strip()
            movie_title = movie_cells[1].text.strip()
            lifetime_gross = movie_cells[2].text.strip()
            distributor = movie_cells[3].text.strip()
            release_date = format_release_date(movie_cells[4].text.strip())
            num_of_theaters = movie_cells[5].text.strip()

            # Obtener el enlace de la película
            movie_link = movie_cells[1].find('a')['href']
            full_movie_url = f"https://www.boxofficemojo.com{movie_link}"

            # Obtener detalles adicionales de la película
            additional_details = get_movie_details(full_movie_url)

            # Agregar información de la película al diccionario
            movie_info = {
                'Rank': rank,
                'Movie Title': movie_title,
                'Lifetime Gross': lifetime_gross,
                'Distributor': distributor,
                'Release Date': release_date,
                'Number of Theaters': num_of_theaters
            }
            movie_info.update(additional_details)  # Añadir los detalles adicionales
            movie_details.append(movie_info)

else:
    print("No se encontró la tabla de películas.")

# Convertir la lista de detalles de películas en un DataFrame de Pandas
movies_df = pd.DataFrame(movie_details).drop_duplicates()

# Limpiar el nombre de la hoja
sheet_name = clean_sheet_name("Top Lifetime Gross")

# Guardar la información en un archivo Excel
with pd.ExcelWriter('top_lifetime_gross_movies.xlsx') as writer:
    movies_df.to_excel(writer, sheet_name=sheet_name, index=False)

print("Datos guardados en 'top_lifetime_gross_movies.xlsx' con éxito.")


C:\Users\Felipe Biatturi\AppData\Local\Temp\ipykernel_18584\3916809294.py:29: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  domestic_distributor = movie_soup.find('span', text='Domestic Distributor')
C:\Users\Felipe Biatturi\AppData\Local\Temp\ipykernel_18584\3916809294.py:33: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  domestic_opening = movie_soup.find('span', text='Domestic Opening')
C:\Users\Felipe Biatturi\AppData\Local\Temp\ipykernel_18584\3916809294.py:37: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  budget = movie_soup.find('span', text='Budget')
C:\Users\Felipe Biatturi\AppData\Local\Temp\ipykernel_18584\3916809294.py:41: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  earliest_release_date = movie_soup.find('span', text='Earliest Release Date')
C:\Users\F

Datos guardados en 'top_lifetime_gross_movies.xlsx' con éxito.
